# 科学技術計算 5 レポート


In [46]:
import numpy as np
import scipy

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond, pinv, lstsq, eig, eigh, eigvals, eigvalsh
from numpy import diag
rng = np.random.default_rng()

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

from typing import List, Optional, Union, Tuple

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces

from matplotlib.pyplot import imshow
from typing import Tuple, List, Optional, Union
from tqdm.auto import tqdm



## 課題05-1

In [47]:
#レイリー商反復について

#関数
def rayleigh_quotient_iteration(A: np.ndarray,mu: float,u,tol=1e-12,n = np.ndarray, maxiter: int = 5000,) -> Tuple[float, np.ndarray]:
    """Rayleigh quotioent iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        u (np.ndarray): n-d initial eigenvector
        maxiter (int, optional): max iterations. Defaults to 50.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    assert len(u) == n
    n = A.shape[0]
    u = u / norm(u)



    for i in range(maxiter):
        # 1. (A - mu*I) が特異になりやすいので ε を足す
        try:
            u_new = solve(A - mu*np.eye(n), u)
        except np.linalg.LinAlgError:
            # 特異になったら微小値でずらす
            u_new = solve(A - (mu + 1e-8)*np.eye(n), u)

        # 2. 正規化
        u_new = u_new / norm(u_new)

        # 3. 新しい Rayleigh 商
        mu_new = u_new.T @ A @ u_new

        # print(f"{i:2d}: mu={mu_new:.12f}, u={u_new}")

        # 4. 収束判定
        if norm(u_new - u) < tol or norm(u_new + u) < tol:
            return mu_new, u_new

        u = u_new
        mu = mu_new
    return mu, u



In [48]:
#ハウスホルダー法
def householder_step( A: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    """k-th step of Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix
        k (int): step

    Returns:
        Tuple[np.ndarray, np.ndarray]: transformed matrix QAQ after k-th step, and the transfrom Q
    """
    assert A.ndim == 2  #2次元であるかチェック
    assert A.shape[0] == A.shape[1] #正方行列かどうかチェック

    n = A.shape[0]
    assert 0 <= k < n - 2

    x = A[k + 1:, k]  # Aのk列目のk+1行目以降の部分を取得(縦の部分)
    y = np.zeros_like(x) #全部ゼロの配列
    # y[0] = - np.sign(A[k + 1, k]) * norm(x)
    y[0] = - np.sign(x[0]) * norm(x)  # ベクトルyの先頭要素を設定

    # print("x", x)
    # print("y", y)

    u = (x - y) / norm(x - y)  # 単位ベクトルuを計算
    H = np.eye(len(x)) - 2 * np.outer(u, u)  # ハウスホルダー行列Hを計算

    Q = np.eye(n)
    Q[k + 1:, k + 1:] = H  # QにHを組み込む

    QAQ = Q @ A @ Q  # 変換後の行列を計算
    return QAQ, Q

def householder_transform(A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: tridiagonal Atri and orthogonal Qhh
                                       so that Qhh A Qhh == Atri
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    Atri = A.copy()
    Qhh = np.eye(n)
    for k in range(n - 2):
        # print(f"{k}-th step")
        Atri, Q = householder_step(Atri, k)
        Qhh = Qhh @ Q  # 直交行列を更新
        # print("Atri\n", Atri, sep='')
        # print("Q\n", Q, sep='')
        # print("Qhh\n", Qhh, sep='')
    return Atri, Qhh

# with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):

#     print("A\n", A, sep='')
#     Atri, Qhh = householder_transform(A)

#     print("tridiagonalized Atri\n", Atri, sep='')
#     print("check Q.T A Q\n", Qhh.T @ A @ Qhh)
#     print("Q.T A Q == Atri", np.allclose(Qhh.T @ A @ Qhh, Atri))

# with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
#     Q, R = np.linalg.qr(Atri)
#     print("Atri\n", Atri, sep='')
#     print("Q\n", Q, sep='')
#     print("R\n", R, sep='')
#     print("RQ\n", R @ Q, sep='')



In [49]:
#ぎぶんす変換
def givens_r_matrix(xp_xq: np.ndarray) -> np.ndarray:
    """generate a matrix of Givens rotation

    Args:
        xp_xq (ndarray): two values xp, xq

    Returns:
        ndarray: 2D rotation matrix
    """
    assert len(xp_xq) == 2

    xp, xq = xp_xq[0], xp_xq[1]

    # denominator = np.sqrt(xp**2 + xq**2)
    denominator = norm(xp_xq)
    c = xp / denominator
    s = -xq / denominator

    R = np.array([
        [c, s],
        [-s, c]
    ])
    return R

def qr_decom_by_givens(Atri_org: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """QR decomposition of a tridiagonal matrix by Givens rotation

    Args:
        Atri_org (np.ndarray): an nxn tridiagonal matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: orthogonal Q and upper traiangular R so that A == QR
    """
    assert len(Atri_org.shape) == 2
    assert Atri_org.shape[0] == Atri_org.shape[1]
    # assert AAtri_org should be tridiagonal
    n = Atri_org.shape[0]

    Atri = Atri_org.copy()
    Q = np.eye(n)

    for k in range(n - 1):
        p, q = k, k + 1
        Rot2x2 = givens_r_matrix(Atri[[p, q]][:, p])
        Atri[[p, q]] = Rot2x2.T @ Atri[[p, q]]
        Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2

    R = Atri
    return Q, R

In [50]:
#QR法関数
def eigh_by_qr_method(A0: np.ndarray, maxiter: int = 1000,) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computing eigenvalues and eigenvectors by QR decompoition

    Args:
        A0 (np.ndarray): nxn matrix
        maxiter (int, optional): max iterations. Defaults to 1000.
        qr_decomp (callable, optional): QR decomposition solver. Defaults to np.linalg.qr.

    Returns:
        Tuple[np.ndarray, np.ndarray]: n-d vector of eigenvalues and orthogonal matrix with eigenvector columns
    """
    assert A0.ndim == 2
    assert A0.shape[0] == A0.shape[1]
    n = A0.shape[0]
    A = A0.copy()

    Qall = np.eye(n)
    diagonals_pre = diag(A)
    superdiagonals_pre = diag(A, k=1)

    for i in range(maxiter):
        Atri, Qhh = householder_transform(A)
        Atri_org = Atri.copy()
        Q, R = qr_decom_by_givens(A)
        A = R @ Q
        Qall = Qall @ Q

        diagonals = diag(A)
        superdiagonals = diag(A, k=1)
        # with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        #     print(f"     diagonals {i:2d}", diagonals)
        #     print(f"superdiagonals {i:2d}", superdiagonals)

        if (np.allclose(diagonals, diagonals_pre)):
            with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
                print("A\n", A, sep='')
            break

        diagonals_pre = diagonals.copy()
        superdiagonals_pre = superdiagonals.copy()

    return diagonals, Qall

# with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
#     print("using numpy QR")
#     d, U = eigh_by_qr_method(A)
#     print("d", d)
#     print("U\n", U)
#     # print("using scipy QR")
#     # d, U = eigh_by_qr_method(A, qr_decomp=scipy.linalg.qr)
#     # print(d)

# with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
#     d_np, U_np = np.linalg.eigh(A)
#     U_np = U_np[:, ::-1]
#     d_np = d_np[::-1]

#     print("d_np", d_np)
#     print("d   ", d)
#     print("d == d_np", np.allclose(d, d_np))

#     print("U_np\n", U_np, sep='')
#     print("U\n", U, sep='')

#     for i in range(len(U)):
#         print(
#             f"column {i}: U[{i}] == U_np[{i}] ?",
#             np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
#         )



In [51]:
#今回のやつ
def eigh_via_qr_rayleigh( A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    # print("A =", A)
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    
    eigenvalues_approx, Qqr  = eigh_by_qr_method(A) #これでAがぎぶんす変換で更新しながら固有値と固有べくとるを出力できたってことかな

    eigenvalues = np.zeros(n)
    eigenvectors = np.zeros((n, n))

    for k in range(n):
        mu0 = eigenvalues_approx[k]
        u0 = Qqr[k, :]

        # print(f"\n--- 第{k}固有値の RQI 開始 ---")
        mu_refined, u_refined = rayleigh_quotient_iteration(A, mu0, u0)

        eigenvalues[k] = mu_refined
        eigenvectors[:, k] = u_refined

    return eigenvalues, eigenvectors


In [ ]:
#main


#実行
for i in tqdm(range(10)):
    n = rng.integers(low = 5, high = 6)
    A = rng.random(size=(n, n))

    #対称行列
    A = A.T @ A
    print("行列 A:")
    print(A)

    d, U = eigh_via_qr_rayleigh(A)
    eigvals_np, eigvecs_np = np.linalg.eigh(A)


    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        d_np, U_np = np.linalg.eigh(A)
        U_np = U_np[:, ::-1]
        d_np = d_np[::-1]

        # print("d_np", d_np)
        # print("d   ", d)
        # print("d == d_np", np.allclose(d, d_np))

        # print("U_np\n", U_np, sep='')
        # print("U\n", U, sep='')

        for i in range(len(U)):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )



  0%|          | 0/10 [00:00<?, ?it/s]

行列 A:
[[0.80771698 0.79797789 1.06757022 0.56177604 0.6174749 ]
 [0.79797789 1.37185497 0.95072007 0.82543466 0.83706223]
 [1.06757022 0.95072007 1.59451295 0.7720422  0.95486298]
 [0.56177604 0.82543466 0.7720422  1.01554921 0.63506816]
 [0.6174749  0.83706223 0.95486298 0.63506816 0.91697771]]
A
[[  4.33684144  -0.00191038   0.56841133   0.25894192   0.14023136]
 [ -0.00191038   0.56590182  -0.00050384  -0.13502507  -0.05130505]
 [  0.56841133  -0.00050384   0.42795887  -0.00079292  -0.00828029]
 [  0.25894192  -0.13502507  -0.00079292   0.33965204  -0.00001019]
 [  0.14023136  -0.05130505  -0.00828029  -0.00001019   0.03625765]]
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 2: U[2] == U_np[2] ? True
column 3: U[3] == U_np[3] ? False
column 4: U[4] == U_np[4] ? False
行列 A:
[[1.27759502 0.68330854 1.19301699 0.91056667 1.08632086]
 [0.68330854 1.03006083 0.76518869 0.70364301 0.37093482]
 [1.19301699 0.76518869 1.48330128 0.56695959 1.06778382]
 [0.91056667 

## 課題05-3